**Index**
+ [x] version 0 : basic
+ [x] version 1 : Query text
+ [x] version 2 : Load Query text

In [1]:
import cx_Oracle as oci

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import numpy as np
import pandas as pd

from datetime import datetime

In [2]:
def get_service(api_name, api_version, scopes, key_file_location):
    credentials = ServiceAccountCredentials.from_json_keyfile_name(key_file_location, scopes=scopes)
    service = build(api_name, api_version, credentials=credentials)
    return service
    
# Define the auth scopes to request.
scope = 'https://www.googleapis.com/auth/analytics.readonly'
key_file_location = './vision API-06a448b64428.json'

# Authenticate and construct service.
service = get_service(api_name='analytics', api_version='v3', scopes=[scope], key_file_location=key_file_location)

# Get a list of all Google Analytics accounts for this user
accounts = service.management().accounts().list().execute()
if accounts.get('items'):
    # Get the first Google Analytics account.
    account = accounts.get('items')[0].get('id')
    # Get a list of all the properties for the first account.
    properties = service.management().webproperties().list(accountId=account).execute()

In [3]:
# 조건1 : 일자
start = "2021-01-01"
end = datetime.today().strftime('%Y-%m-%d')
start_date_info = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='W')]
end_date_info   = [d.strftime('%Y-%m-%d') for d in pd.date_range(start, end, freq='W')+ pd.offsets.Day(6)]

start_date_week = [d.strftime('%Y-%W') for d in pd.date_range(start, end, freq='W')+ pd.offsets.Day(6)]

In [16]:
len(start_date_info), len(end_date_info), len(start_date_week)

(23, 23, 23)

## 1.Acquisition

In [24]:
##########
#1.Acquisition.txt
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/1.Acquisition_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df1 = cursor.fetchall()[1:]

df1 = pd.DataFrame(df1, columns = ['MONTH', 'App install', 'App install : paid'])
df1 = df1.set_index('MONTH').T.fillna(0)

In [25]:
df1

MONTH,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23,2021-53
App install,409.0,59.0,94.0,12974.0,3280.0,1680.0,288.0,252.0,251.0,288.0,...,368.0,614.0,299.0,425.0,204.0,339.0,268.0,215.0,57.0,3.0
App install : paid,55.0,15.0,20.0,8188.0,2026.0,1041.0,121.0,122.0,153.0,161.0,...,245.0,460.0,180.0,276.0,111.0,218.0,171.0,145.0,36.0,0.0


In [26]:
# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:browser'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df1_ga = pd.DataFrame( columns =column_name)
df1_ga['date'] = ''

# 결과 수집+정리
for i in range(0, len(start_date_info)):
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_week[i]
        df1_ga = pd.concat([df1_ga, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df1_ga['gubun'] = 'APP activation'

# 임시자료 추가(2021년 1월 미수집에 따른)
df_temp1 = pd.DataFrame({"browser":['Android Webview'], "users":[0], "date":['2021-01'], "gubun":['APP activation']})
df_temp2 = pd.DataFrame({"browser":['Android Webview'], "users":[0], "date":['2021-02'], "gubun":['APP activation']})
df_temp3 = pd.DataFrame({"browser":['Android Webview'], "users":[0], "date":['2021-03'], "gubun":['APP activation']})
df_temp4 = pd.DataFrame({"browser":['Android Webview'], "users":[0], "date":['2021-04'], "gubun":['APP activation']})
df_temp5 = pd.DataFrame({"browser":['Android Webview'], "users":[0], "date":['2021-05'], "gubun":['APP activation']})
df_temp = pd.concat([df_temp1,df_temp2,df_temp3,df_temp4,df_temp5 ])

df1_ga = df1_ga.append(df_temp, ignore_index = True)

# 필요한 자료만 필터링
df1_ga = df1_ga[(df1_ga.browser == 'Android Webview' )|(df1_ga.browser == 'Safari (in-app)')]

# header name 변경
df1_ga.rename(columns = {'browser' : 'gubun2', 'users' : 'value'}, inplace = True)
df1_ga = df1_ga.reset_index(drop = True)

# 월 정렬
df1_ga['date'] = df1_ga['date'].str[:7]

# pivot table
df_MTLY = pd.pivot_table(df1_ga, index = ['gubun2'], values = 'value', columns = 'date', aggfunc = np.sum).fillna(0)

# total값 추가가공
df_MTLY = df_MTLY.T
df_MTLY['APP activation : total'] = df_MTLY['Android Webview'].astype(int)+ df_MTLY['Safari (in-app)'].astype(int)
df_MTLY.rename(columns = {'Android Webview' : 'Android', 'Safari (in-app)' : 'iOS'}, inplace = True)
df_MTLY = df_MTLY.T

# 기준 테이블 작성/order 수정
df_order = pd.DataFrame()
df_order['gubun2'] = ['APP activation : total', 'Android', 'iOS']

df_MTLY= pd.merge(df_order, df_MTLY, on ='gubun2', how ='left')

df_MTLY.set_index(df_MTLY.gubun2,inplace = True)
del df_MTLY['gubun2']

In [27]:
df_MTLY

,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,...,2021-14,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23
gubun2,,,,,,,,,,,,,,,,,,,,,
APP activation : total,0,0,0,0,0,6856,9322,4827,4117,5403,...,2381,17326,22619,27205,35078,15719,18761,20133,20026,5535
Android,0,0,0,0,0,4938,6768,3625,3178,4540,...,1819,15962,20598,24887,32182,12984,14471,15825,15618,4582
iOS,0,0,0,0,0,1918,2554,1202,939,863,...,562,1364,2021,2318,2896,2735,4290,4308,4408,953


In [5]:
#############
# Weekly, APP activation
#####

# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:browser'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:users'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df1_ga = pd.DataFrame( columns =column_name)
df1_ga['date'] = ''

# 결과 수집+정리
for i in range(0, len(start_date_info)):
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_week[i]
        df1_ga = pd.concat([df1_ga, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df1_ga['gubun'] = 'APP activation'

# 필요한 자료만 필터링
df1_ga = df1_ga[(df1_ga.browser == 'Android Webview' )|(df1_ga.browser == 'Safari (in-app)')]

# header name 변경
df1_ga.rename(columns = {'browser' : 'gubun2', 'users' : 'value'}, inplace = True)

df1_MTLY = pd.pivot_table(df1_ga, index = ['gubun2'], values = 'value', columns = 'date', aggfunc = np.sum).fillna(0)

In [6]:
df1_MTLY

date,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2021-13,2021-14,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23
gubun2,,,,,,,,,,,,,,,,,,
Android Webview,4938,6768,3625,3178,4540,2619,2350,2286,1819,15962,20598,24887,32182,12984,14471,15825,15618,4565
Safari (in-app),1918,2554,1202,939,863,803,787,630,562,1364,2021,2318,2896,2735,4290,4308,4408,951


In [7]:

# 조건2 : 필터링 항목(dimensions)
dimensions_value = 'ga:channelGrouping'

# 조건3 : 측정 단위(metrics)
metrics_value = 'ga:sessions'

# 수집내용 정의
column_name = (dimensions_value +','+ metrics_value).replace('ga:', '').split(",")
df2_ga = pd.DataFrame( columns =column_name)
df2_ga['date'] = ''

# 결과 수집+정리
for i in range(0, len(start_date_info)):
    try:
        result = service.data().ga().get(ids='ga:236972743',
                                         start_date=start_date_info[i], end_date=end_date_info[i], 
                                         dimensions=dimensions_value, metrics=metrics_value).execute()
        df_temp = pd.DataFrame(result['rows'], columns =column_name)
        df_temp['date'] = start_date_week[i]
        df2_ga = pd.concat([df2_ga, df_temp])
    except:
        pass

# 자료의 상위속성 추가
df2_ga['gubun'] = 'ALL Activation'

# header name 변경
df2_ga.rename(columns = {'channelGrouping' : 'gubun2', 'sessions' : 'value'}, inplace = True)
df2_ga = df2_ga.reset_index(drop = True)

df2_MTLY = pd.pivot_table(df2_ga, index = ['gubun2'], values = 'value', columns = 'date', aggfunc = np.sum).fillna(0)

# 기준 테이블 작성/order 수정
df_order = pd.DataFrame()
df_order['gubun2'] = ['Display', 'Direct', 'Organic Search', 'Social', 'Paid Search', 'Referral', '(Other)']

df2_MTLY= pd.merge(df_order, df2_MTLY, on ='gubun2', how ='left')

df2_MTLY.set_index(df2_MTLY.gubun2,inplace = True)
del df2_MTLY['gubun2']

# df2_MTLY = df2_MTLY.reset_index(drop = False)

In [8]:
df2_MTLY

,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2021-13,2021-14,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23
gubun2,,,,,,,,,,,,,,,,,,
Display,0,0,0,3,8416,533,227,134,94,2619,3128,3510,3542,3685,2969,3178,694,0
Direct,14036,23309,8548,7687,5479,5558,4831,4628,3305,6732,8029,5359,6307,3683,4481,4720,4327,1475
Organic Search,414,1148,966,1170,1011,1082,1340,1579,1352,2122,3131,3705,6154,6147,5768,4085,4548,1161
Social,394,773,904,723,623,578,618,780,666,1490,1077,1147,1832,1968,1700,802,254,0
Paid Search,0,0,311,20,2,0,0,0,0,0,33,171,187,203,154,247,51,0
Referral,51,101,105,112,116,107,103,77,79,490,425,242,275,159,112,141,50,0
(Other),0,0,1,102,173,961,469,447,222,18612,36318,81792,76658,24651,31020,34324,21880,3392


In [9]:
Acquisition = pd.concat([df1, df1_MTLY, df2_MTLY]).fillna(0)
Acquisition = Acquisition.apply(pd.to_numeric)
del Acquisition['2021-53']
Acquisition['증가율(%)'] = round((Acquisition[Acquisition.columns[-1]] - Acquisition[Acquisition.columns[-2]])/Acquisition[Acquisition.columns[-2]]*100,2)

In [10]:
Acquisition

,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23,증가율(%)
App install,409.0,59.0,94.0,12974.0,3280.0,1680.0,288.0,252.0,251.0,288.0,...,368.0,614.0,299.0,425.0,204.0,339.0,268.0,215.0,57.0,-73.49
App install : paid,55.0,15.0,20.0,8188.0,2026.0,1041.0,121.0,122.0,153.0,161.0,...,245.0,460.0,180.0,276.0,111.0,218.0,171.0,145.0,36.0,-75.17
Android Webview,0.0,0.0,0.0,0.0,4938.0,6768.0,3625.0,3178.0,4540.0,2619.0,...,15962.0,20598.0,24887.0,32182.0,12984.0,14471.0,15825.0,15618.0,4433.0,-71.62
Safari (in-app),0.0,0.0,0.0,0.0,1918.0,2554.0,1202.0,939.0,863.0,803.0,...,1364.0,2021.0,2318.0,2896.0,2735.0,4290.0,4308.0,4408.0,921.0,-79.11
Display,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,8416.0,533.0,...,2619.0,3128.0,3510.0,3542.0,3685.0,2969.0,3178.0,694.0,0.0,-100.00
Direct,0.0,0.0,0.0,0.0,14036.0,23309.0,8548.0,7687.0,5479.0,5558.0,...,6732.0,8029.0,5359.0,6307.0,3683.0,4481.0,4720.0,4327.0,1475.0,-65.91
Organic Search,0.0,0.0,0.0,0.0,414.0,1148.0,966.0,1170.0,1011.0,1082.0,...,2122.0,3131.0,3705.0,6154.0,6147.0,5768.0,4085.0,4548.0,1161.0,-74.47
Social,0.0,0.0,0.0,0.0,394.0,773.0,904.0,723.0,623.0,578.0,...,1490.0,1077.0,1147.0,1832.0,1968.0,1700.0,802.0,254.0,0.0,-100.00
Paid Search,0.0,0.0,0.0,0.0,0.0,0.0,311.0,20.0,2.0,0.0,...,0.0,33.0,171.0,187.0,203.0,154.0,247.0,51.0,0.0,-100.00
Referral,0.0,0.0,0.0,0.0,51.0,101.0,105.0,112.0,116.0,107.0,...,490.0,425.0,242.0,275.0,159.0,112.0,141.0,50.0,0.0,-100.00


In [28]:
##########
# 2.Activation
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/2.Activation_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df2 = cursor.fetchall()[1:]
# df1 = pd.DataFrame(df1)
df2 = pd.DataFrame(df2, columns = ['MONTH', '신규가입', '신규가입-첫구매완료', '신규가입-신규대비(%)', '누적-회원수','누적-구매',
                                   '수신동의', '수신동의-SMS동의', '수신동의-PUSH동의', '수신동의-EMIL동의'])
Activation = df2.set_index('MONTH').T.fillna(0)

In [29]:
Activation

MONTH,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23,2021-53
신규가입,428.0000,64.0000,103.0000,14265.0000,3685.0000,1906.0000,411.0000,362.0000,373.0000,435.000,...,667.0000,1334.0000,1529.0000,1804.000,1031.0000,1551.0000,1490.0000,1016.0000,253.0000,5.0
신규가입-첫구매완료,258.0000,37.0000,68.0000,11708.0000,2945.0000,1310.0000,253.0000,247.0000,278.0000,338.000,...,482.0000,883.0000,595.0000,1086.000,503.0000,844.0000,702.0000,581.0000,122.0000,3.0
신규가입-신규대비(%),0.6028,0.5781,0.6602,0.8208,0.7992,0.6873,0.6156,0.6823,0.7453,0.777,...,0.7226,0.6619,0.3891,0.602,0.4879,0.5442,0.4711,0.5719,0.4822,0.6
누적-회원수,718.0000,782.0000,885.0000,15150.0000,18835.0000,20741.0000,21152.0000,21514.0000,21887.0000,22322.000,...,24017.0000,25351.0000,26880.0000,28684.000,29715.0000,31266.0000,32756.0000,33772.0000,34025.0000,34030.0
누적-구매,5131.0000,5238.0000,5560.0000,110706.0000,52856.0000,44318.0000,24029.0000,22964.0000,24174.0000,34547.000,...,40519.0000,59256.0000,50976.0000,80153.000,49238.0000,70657.0000,71805.0000,72127.0000,19642.0000,37.0
수신동의,389.0000,428.0000,480.0000,7419.0000,9094.0000,9870.0000,10031.0000,10205.0000,10365.0000,10576.000,...,11372.0000,12049.0000,12918.0000,13943.000,14512.0000,15336.0000,16206.0000,16755.0000,16891.0000,16895.0
수신동의-SMS동의,9888.0000,10753.0000,12644.0000,16375.0000,16867.0000,16867.0000,16867.0000,16867.0000,16867.0000,16867.000,...,16867.0000,16867.0000,16867.0000,16867.000,16867.0000,16867.0000,16867.0000,16867.0000,16867.0000,16867.0
수신동의-PUSH동의,228.0000,246.0000,270.0000,3092.0000,3780.0000,4088.0000,4148.0000,4218.0000,4289.0000,4379.000,...,4692.0000,4884.0000,4991.0000,5173.000,5258.0000,5402.0000,5529.0000,5646.0000,5677.0000,5679.0
수신동의-EMIL동의,356.0000,395.0000,444.0000,6964.0000,8512.0000,9225.0000,9375.0000,9527.0000,9664.0000,9842.000,...,10538.0000,11153.0000,11971.0000,12927.000,13466.0000,14242.0000,15067.0000,15571.0000,15696.0000,15699.0


In [13]:
##########
# 3.Retention-Daily
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/3.Retention-Daily_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df3 = cursor.fetchall()[1:]
df3 = pd.DataFrame(df3, columns = ['MONTH', ('Daily PU', ''), ('Daily PU', 'NPU'),  ('Daily PU', 'RPU'),  ('Daily PU', '비회원'), 
                                   'Daily 결제건수(Total)', 'Daily 결제건수(NPU)', 'Daily 결제건수(RPU)',
                                   'Daily PU_쿠폰사용(Total)', 'Daily PU_쿠폰사용(NPU)', 'Daily PU_쿠폰사용(RPU)',
                                   'Daily 결제건수_쿠폰사용(Total)', 'Daily 결제건수_쿠폰사용(NPU)', 'Daily 결제건수_쿠폰사용(RPU)'])
df3_1 = df3.set_index('MONTH').T.fillna(0)

In [14]:
##########
# 3.Retention-Daily
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/3.Retention-Monthly_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df3 = cursor.fetchall()[1:]
df3 = pd.DataFrame(df3, columns = ['MONTH', ('Monthly PU', ''), ('Monthly PU', 'NPU'),  ('Monthly PU', 'RPU'),  ('Monthly PU', '비회원'), 
                                   'Monthly 결제건수(Total)', 'Monthly 결제건수(NPU)', 'Monthly 결제건수(RPU)',
                                   'Monthly PU_쿠폰사용(Total)', 'Monthly PU_쿠폰사용(NPU)', 'Monthly PU_쿠폰사용(RPU)',
                                   'Monthly 결제건수_쿠폰사용(Total)', 'Monthly 결제건수_쿠폰사용(NPU)', 'Monthly 결제건수_쿠폰사용(RPU)'])
df3_2 = df3.set_index('MONTH').T.fillna(0)

In [15]:
##########
# 4.Retention-Freq
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/3.Retention-Freq_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df3 = cursor.fetchall()[1:]
# df4 = pd.DataFrame(df4)
df3 = pd.DataFrame(df3, columns = ['MONTH', '구매빈도(합계)', '구매빈도(신규)', '구매빈도(유보)', '구매빈도(시도)', '구매빈도(정착)',
                                   '구매빈도(이탈)', '구매빈도(휴면)'])
df3_3 = df3.set_index('MONTH').T.fillna(0)

In [16]:
##########
# 3.Retention-Freq
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/3.Retention-timediff_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df3 = cursor.fetchall()[1:]
# df3 = pd.DataFrame(df3)
df3 = pd.DataFrame(df3, columns = ['MONTH', '구매빈도v3(누계)', '구매빈도v3(월별)'])
df3_4 = df3.set_index('MONTH').T.fillna(0)

In [17]:
Retention = pd.concat([df3_1, df3_2, df3_3, df3_4])
Retention = Retention.apply(pd.to_numeric)
Retention['증가율(%)'] = round((Retention[Retention.columns[-1]] - Retention[Retention.columns[-2]])/Retention[Retention.columns[-2]]*100,2)

In [18]:
round(Retention['증가율(%)'].mean(),2)

-84.73

In [19]:
Retention

MONTH,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23,2021-53,증가율(%)
"(Daily PU, )",115.00,168.00,105.00,9973.00,3291.00,2365.00,706.00,682.00,597.00,710.00,...,1609.00,1034.00,1779.00,804.00,1169.00,1045.00,1032.00,282.00,1.0,-99.65
"(Daily PU, NPU)",104.00,133.00,79.00,9872.00,2996.00,2004.00,401.00,355.00,361.00,408.00,...,1146.00,691.00,1274.00,557.00,857.00,720.00,637.00,151.00,1.0,-99.34
"(Daily PU, RPU)",15.00,47.00,34.00,277.00,339.00,406.00,324.00,335.00,241.00,325.00,...,509.00,379.00,567.00,267.00,344.00,352.00,431.00,132.00,0.0,-100.00
"(Daily PU, 비회원)",1.00,0.00,0.00,0.00,1.00,2.00,3.00,2.00,3.00,3.00,...,3.00,1.00,10.00,14.00,19.00,21.00,18.00,2.00,0.0,-100.00
Daily 결제건수(Total),130.00,186.00,131.00,10228.00,3354.00,2470.00,746.00,717.00,620.00,767.00,...,1713.00,1140.00,1932.00,918.00,1329.00,1246.00,1232.00,312.00,1.0,-99.68
Daily 결제건수(NPU),104.00,133.00,79.00,9872.00,2996.00,2004.00,401.00,355.00,361.00,408.00,...,1146.00,691.00,1274.00,557.00,857.00,720.00,637.00,151.00,1.0,-99.34
Daily 결제건수(RPU),26.00,53.00,52.00,356.00,358.00,466.00,345.00,362.00,259.00,359.00,...,567.00,449.00,658.00,361.00,472.00,526.00,595.00,161.00,0.0,-100.00
Daily PU_쿠폰사용(Total),4.00,35.00,25.00,79.00,218.00,322.00,254.00,295.00,184.00,324.00,...,158.00,93.00,162.00,65.00,100.00,117.00,158.00,45.00,0.0,-100.00
Daily PU_쿠폰사용(NPU),2.00,8.00,3.00,8.00,31.00,44.00,27.00,17.00,16.00,89.00,...,40.00,6.00,10.00,1.00,0.00,3.00,4.00,1.00,0.0,-100.00
Daily PU_쿠폰사용(RPU),2.00,28.00,22.00,71.00,187.00,278.00,227.00,278.00,169.00,238.00,...,119.00,87.00,152.00,64.00,100.00,114.00,154.00,44.00,0.0,-100.00


In [20]:
##########
# 4.Revenue-GMV
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/4.Revenue-GMV_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df4 = cursor.fetchall()[1:]
# df4 = pd.DataFrame(df4)
df4 = pd.DataFrame(df4, columns = ['MONTH', '주문_Total(A+B+C)'  , '주문_Total_NPU',       '주문_Total_RPU',
                                            '결제금액_Total(A)'  , '결제금액_Total_NPU',   '결제금액_Total_RPU', 
                                            '제품할인_Total(B)'  , '제품할인_Total_NPU',   '제품할인_Total_RPU', 
                                            '포인트사용_Total(C)', '포인트사용_Total_NPU', '포인트사용_Total_RPU', 
                                            '포인트적립_Total'   , '포인트적립_Total_NPU', '포인트적립_Total_RPU' ])
df4_1 = df4.set_index('MONTH').T.fillna(0)

In [21]:
##########
# 4.Revenue-기여효과
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/4.Revenue-GMV_기여효과_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df4 = cursor.fetchall()[1:]
# df4 = pd.DataFrame(df4)
df4 = pd.DataFrame(df4, columns = ['MONTH', 'GMV_ALL',      'GMV_쿠폰',      'GMV_포인트',      'GMV_쿠N포',
                                            '결제건수_ALL', '결제건수_쿠폰', '결제건수_포인트', '결제건수_쿠N포'
                                  ])
df4_2 = df4.set_index('MONTH').T.fillna(0)


In [22]:
##########
# 4.Revenue-MC
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/4.Revenue-MC_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df4 = cursor.fetchall()[1:]
# df4 = pd.DataFrame(df4)
df4 = pd.DataFrame(df4, columns = ['MONTH', 'Cost(Total)'  , 'Cost(ratio)'])
df4_3 = df4.set_index('MONTH').T.fillna(0)

In [23]:
Revenue = pd.concat([df4_1, df4_2, df4_3])
Revenue = Revenue.apply(pd.to_numeric)
del Revenue['2021-53']
Revenue['증가율(%)'] = round((Revenue[Revenue.columns[-1]] - Revenue[Revenue.columns[-2]])/Revenue[Revenue.columns[-2]]*100,2)

In [24]:
Revenue

MONTH,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23,증가율(%)
주문_Total(A+B+C),10454.0000,12494.0000,11151.0000,743431.0000,266287.0000,191129.0000,52914.000,47525.0000,48921.0000,72094.0000,...,88186.0000,146247.0000,98650.0000,178751.000,79701.0000,108565.0000,105122.0000,111500.0000,30485.0000,-72.66
주문_Total_NPU,7667.0000,8604.0000,6057.0000,709406.0000,237243.0000,154597.0000,25992.000,22483.0000,28523.0000,34412.0000,...,45380.0000,95824.0000,59134.0000,115006.000,44629.0000,63683.0000,54334.0000,53274.0000,13211.0000,-75.20
주문_Total_RPU,2787.0000,3890.0000,5094.0000,34025.0000,29044.0000,36532.0000,26922.000,25042.0000,20398.0000,37682.0000,...,42806.0000,50423.0000,39516.0000,63745.000,35072.0000,44882.0000,50788.0000,58226.0000,17274.0000,-70.33
결제금액_Total(A),5156.0000,5288.0000,5575.0000,119238.0000,56666.0000,46900.0000,24471.000,23429.0000,24434.0000,34772.0000,...,40979.0000,60261.0000,51466.0000,80801.000,49418.0000,70869.0000,71985.0000,72310.0000,19677.0000,-72.79
결제금액_Total_NPU,3466.0000,3421.0000,2827.0000,109901.0000,48583.0000,35893.0000,12834.000,11770.0000,14932.0000,17196.0000,...,20513.0000,38283.0000,30740.0000,52450.000,25674.0000,39621.0000,34817.0000,32713.0000,8104.0000,-75.23
결제금액_Total_RPU,1690.0000,1867.0000,2748.0000,9337.0000,8083.0000,11007.0000,11637.000,11659.0000,9502.0000,17576.0000,...,20466.0000,21978.0000,20726.0000,28351.000,23744.0000,31248.0000,37168.0000,39597.0000,11573.0000,-70.77
제품할인_Total(B),4099.0000,5245.0000,4548.0000,489394.0000,166680.0000,113209.0000,22652.000,20044.0000,20953.0000,34423.0000,...,43293.0000,76906.0000,41403.0000,89796.000,23147.0000,34501.0000,30394.0000,36810.0000,10151.0000,-72.42
제품할인_Total_NPU,3074.0000,3505.0000,2369.0000,468407.0000,148812.0000,92890.0000,9923.000,8294.0000,11476.0000,15146.0000,...,22482.0000,51208.0000,24644.0000,57198.000,12704.0000,21620.0000,17455.0000,18797.0000,4676.0000,-75.12
제품할인_Total_RPU,1025.0000,1740.0000,2179.0000,20987.0000,17868.0000,20319.0000,12729.000,11750.0000,9477.0000,19277.0000,...,20811.0000,25698.0000,16759.0000,32598.000,10443.0000,12881.0000,12939.0000,18013.0000,5475.0000,-69.61
포인트사용_Total(C),1199.0000,1961.0000,1028.0000,134800.0000,42941.0000,31020.0000,5791.000,4051.0000,3534.0000,2899.0000,...,3914.0000,9080.0000,5780.0000,8154.000,7135.0000,3196.0000,2742.0000,2380.0000,656.0000,-72.44


In [25]:
##########
# 5.Referral
####

# query 열기
query = "C:/Users/MAEIL/Desktop/셀렉스몰/18. query(AARRR)/AARRR_query_v2/weekly/5.Referral_weekly_v2.3.txt"

# DB접속 : 접속정보 읽기
con_text = open("C:/Users/MAEIL/conn_SELEX.txt", 'r', encoding='utf8')
con_text = con_text.read()

# DB 접속 : 실제 접속
conn = oci.connect(con_text[1:], encoding='UTF-8', nencoding='UTF-8')

f = open(query, 'r')
line = f.read()
f.close()

## query 실행( SQL문 실행 메모리 영역에서 진행)
cursor = conn.cursor()
cursor.execute(line)
df5 = cursor.fetchall()[1:]
# df5 = pd.DataFrame(df5)
df5 = pd.DataFrame(df5, columns = ['MONTH', '추천인_유입수', '추천자_발생수', '추천자_GMV', '추천자_평균매출','리뷰수', '제품평점'])
df5 = df5.set_index('MONTH').T.fillna(0)

In [26]:
Referral = df5
Referral = Referral.apply(pd.to_numeric)
del Referral['2021-53']
Referral['증가율(%)'] = round((Referral[Referral.columns[-1]] - Referral[Referral.columns[-2]])/Referral[Referral.columns[-2]]*100,2)

In [27]:
Referral

MONTH,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,...,2021-15,2021-16,2021-17,2021-18,2021-19,2021-20,2021-21,2021-22,2021-23,증가율(%)
추천인_유입수,0.0,0.0,7.0,3957.00,1220.00,405.00,0.00,0.00,0.00,0.00,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
추천자_발생수,0.0,0.0,10.0,13301.00,3465.00,1410.00,0.00,0.00,0.00,0.00,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
추천자_GMV,0.0,0.0,0.0,701986.00,194125.00,92823.00,0.00,0.00,0.00,0.00,...,247.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
추천자_평균매출,0.0,0.0,0.0,52.78,56.02,65.83,0.00,0.00,0.00,0.00,...,247.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN
리뷰수,38.0,45.0,48.0,284.00,1386.00,1031.00,611.00,420.00,324.00,239.00,...,289.00,530.00,356.00,361.00,176.00,278.00,266.00,262.00,69.00,-73.66
제품평점,5.0,5.0,5.0,4.98,4.98,4.98,4.95,4.96,4.97,4.97,...,4.97,4.96,4.99,4.97,4.99,4.95,4.97,4.95,4.97,0.40


In [30]:
##########
# Excel 파일 생성(using XlsxWriter)
###

date_today = datetime.today().strftime('%Y%m%d')
date_today = date_today[2:]
excel_name = date_today +'_AARRR_Weekly.xlsx'
writer = pd.ExcelWriter(excel_name, engine='xlsxwriter')
 
Acquisition.to_excel(writer, sheet_name='1.Acquisition')
Activation.to_excel(writer, sheet_name='2.Activation')
Retention.to_excel(writer, sheet_name='2.Retention')
Revenue.to_excel(writer, sheet_name='4.Revenue')
Referral.to_excel(writer, sheet_name='5.Referral')

# Close the Pandas Excel writer and output the Excel file.
writer.save()